# What is LangGraph?

LangGraph is a framework developed by LangChain to manage the control flow of applications that integrate an LLM.

## Is LangGraph different from LangChain ?
LangChain provides a standard interface to interact with models and other components, useful for retrieval, LLM calls and tools calls. The classes from LangChain might be used in LangGraph, but do not HAVE to be used.

The packages are different and can be used in isolation, but, in the end, all resources you will find online use both packages hand in hand.

## When should I use LangGraph ?
### Control vs freedom
When designing AI applications, you face a fundamental trade-off between control and freedom:

- **Freedom** gives your LLM more room to be creative and tackle unexpected problems.
- **Control** allows you to ensure predictable behavior and maintain guardrails.
  
Code Agents, like the ones you can encounter in smolagents, are very free. They can call multiple tools in a single action step, create their own tools, etc. However, this behavior can make them less predictable and less controllable than a regular Agent working with JSON!

`LangGraph` is on the other end of the spectrum, it shines when you need **“Control” on the execution of your agent**.

LangGraph is particularly valuable when you need **Control over your applications**. It gives you the tools to build an application that follows a predictable process while still leveraging the power of LLMs.

Put simply, if your application involves a series of steps that need to be orchestrated in a specific way, with decisions being made at each junction point, **LangGraph provides the structure you need**.

As an example, let’s say we want to build an LLM assistant that can answer some questions over some documents.

Since LLMs understand text the best, before being able to answer the question, you will need to convert other complex modalities (charts, tables) into text. However, that choice depends on the type of document you have!

This is a branching that I chose to represent as follow :

![](../../image/flow_langgraph.png)


While this branching is deterministic, you can also design branching that are conditioned on the output of an LLM making them undeterministic.

The key scenarios where LangGraph excels include:

- **Multi-step reasoning processes** that need explicit control on the flow
- **Applications requiring persistence of state** between steps
- **Systems that combine deterministic logic with AI capabilities**
- **Workflows that need human-in-the-loop interventions**
- **Complex agent architectures** with multiple components working together

In essence, whenever possible, **as a human**, design a flow of actions based on the output of each action, and decide what to execute next accordingly. In this case, LangGraph is the correct framework for you!

LangGraph is, in my opinion, the most production-ready agent framework on the market.

## How does LangGraph work?

At its core, `LangGraph` uses a directed graph structure to define the flow of your application:

- **Nodes** represent individual processing steps (like calling an LLM, using a tool, or making a decision).
- **Edges** define the possible transitions between steps.
- **State** is user defined and maintained and passed between nodes during execution. When deciding which node to target next, this is the current state that we look at.

## How is it different from regular python? Why do I need LangGraph?
While technically true, LangGraph offers some advantages over vanilla Python for building complex systems. You could build the same application without LangGraph, but it builds easier tools and abstractions for you.

It includes states, visualization, logging (traces), built-in human-in-the-loop, and more.


## Building Blocks of LangGraph
To build applications with LangGraph, you need to understand its core components. Let’s explore the fundamental building blocks that make up a LangGraph application.


An application in LangGraph starts from an entrypoint, and depending on the execution, the flow may go to one function or another until it reaches the END.

![](../../image/application.png)


### 1. State
State is the central concept in LangGraph. It represents all the information that flows through your application.

In [ ]:
from typing_extensions import TypedDict

class State(TypedDict):
    graph_state: str

The state is **User defined**, hence the fields should carefully be crafted to contain all data needed for decision-making process!

> 💡 Tip: Think carefully about what information your application needs to track between steps.

### 2.Node
Nodes are python functions. Each node:

- Takes the state as input
- Performs some operation
- Returns updates to the state

In [ ]:
def node_1(state):
    print("---Node 1---")
    return {"graph_state": state['graph_state'] +" I am"}

def node_2(state):
    print("---Node 2---")
    return {"graph_state": state['graph_state'] +" happy!"}

def node_3(state):
    print("---Node 3---")
    return {"graph_state": state['graph_state'] +" sad!"}

For example, Nodes can contain:

- **LLM calls**: Generate text or make decisions
- **Tool calls**: Interact with external systems
- **Conditional logic**: Determine next steps
- **Human intervention**: Get input from users

> 💡 Info: Some nodes necessary for the whole workflow like START and END exist from langGraph directly.

### 3.Edges
**Edges** connect nodes and define the possible paths through your graph:



In [ ]:
import random
from typing import Literal

def decide_mood(state) -> Literal["node_2", "node_3"]:
    
    # Often, we will use state to decide on the next node to visit
    user_input = state['graph_state'] 
    
    # Here, let's just do a 50 / 50 split between nodes 2, 3
    if random.random() < 0.5:

        # 50% of the time, we return Node 2
        return "node_2"
    
    # 50% of the time, we return Node 3
    return "node_3"


Edges can be:

- **Direct**: Always go from node A to node B
- **Conditional**: Choose the next node based on the current state

### 4. StateGraph

The **StateGraph** is the container that holds your entire agent workflow:



In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END

# Build graph
builder = StateGraph(State)
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)

# Logic
builder.add_edge(START, "node_1")
builder.add_conditional_edges("node_1", decide_mood)
builder.add_edge("node_2", END)
builder.add_edge("node_3", END)

# Add
graph = builder.compile()

In [ ]:
Which can then be visualized! But most importantly, invoked:





In [ ]:
# View
display(Image(graph.get_graph().draw_mermaid_png()))

# Invoke
graph.invoke({"graph_state" : "Hi, this is Lance."})
